In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
import os
from sklearn import preprocessing
from sklearn.externals import joblib
import multiprocessing as mp

#import PatternVectorizer and SimpleClassifier
from pattern_classifier import  SimpleClassifier, PatternVectorizer

In [2]:
cls_persistence = 'data/simple_classifier_model.pkl.compressed'
pv_persistence = 'data/pattern_vectorizer.pkl.compressed'

cls = joblib.load(cls_persistence)
pv = joblib.load(pv_persistence)

## Load tweets

In [3]:
file_dir = 'mention_recent/'
col = ['id', 'screen', 'datetime', 'text']
user_tweet_list = [pd.read_csv(file_dir+file, delimiter = '\t', names = col) for file in os.listdir(file_dir)]

## Query Emotion

In [20]:
def emotion_query(user_tweets):
    documentPatternVectors = pv.transform(user_tweets.text)

    # using two emotion, guess the one with the smallest rank
    Y_GUESS_2 = cls.get_top_emotion_ambiguous(documentPatternVectors, ascending=True, n=2)

    Guess_array = np.array(Y_GUESS_2)

    user_tweets['emotion1'] = Guess_array[:,0]
    user_tweets['emotion2'] = Guess_array[:,1]
    user_tweets['ambiguous'] = Guess_array[:,2]
    
    return user_tweets

In [21]:
pool = mp.Pool(processes=mp.cpu_count()-1)
multi_res = [pool.apply_async(emotion_query, (user_tweets,)) for user_tweets in user_tweet_list]

emo_user_tweet_list = [res.get() for res in multi_res]

,id,screen,datetime,text,emotion1,emotion2,ambiguous
0,freakpatrol,freakpatrol,2015-11-19 07:54:19,9:50 in the morning!! the fuck is that about!!!,surprise,joy,false
1,freakpatrol,freakpatrol,2015-11-19 07:12:13,The heat works and I got free candy today I am...,joy,surprise,false
2,freakpatrol,freakpatrol,2015-11-18 16:30:39,Had to turn off the heat it's 30 something out...,anger,fear,false
3,freakpatrol,freakpatrol,2015-11-18 10:17:31,the power is flickering again !!!!!,joy,fear,false
4,freakpatrol,freakpatrol,2015-11-18 10:05:40,just dropped over syria. like it's not funny a...,surprise,anger,false
5,freakpatrol,freakpatrol,2015-11-18 10:04:28,i had to unfollow this really nice french girl...,anger,surprise,false
6,freakpatrol,freakpatrol,2015-11-18 10:02:49,lights went out in the house for a full second...,fear,joy,false
7,freakpatrol,freakpatrol,2015-11-15 06:26:53,if a mentally ill person doesn't want to talk ...,surprise,anger,false
8,freakpatrol,freakpatrol,2015-11-14 03:48:12,WORST DAY EVER OMFG,fear,anger,true
9,freakpatrol,freakpatrol,2015-11-14 00:49:26,I try so hard in everything that I do why do p...,anger,surprise,false
